In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder

In [3]:
df = pd.read_csv('data/20220208_Final_result.csv')

# Data exploration

### Remove Null Price rows and change type to float

In [4]:
df = df[pd.to_numeric(df['Price'], errors='coerce').notnull()]

In [5]:
df = df.astype({"Price": float}, errors='raise')

### Remove duplicate ID

In [6]:
df['Immoweb ID'].value_counts()

9691876    4
9662650    3
9597143    3
8237073    3
9687272    3
          ..
9623733    1
9245768    1
9606238    1
9695276    1
7921699    1
Name: Immoweb ID, Length: 13893, dtype: int64

In [7]:
df = df.drop_duplicates(['Immoweb ID'], keep='last')
df['Immoweb ID'].value_counts()

9729720    1
9678715    1
9697071    1
9557609    1
9678634    1
          ..
9518030    1
9622950    1
9478102    1
9620790    1
7921699    1
Name: Immoweb ID, Length: 13893, dtype: int64

### Remove subtype 

In [8]:
df = df[df['property sub-type'] != "APARTMENT_BLOCK"]
df['property sub-type'].value_counts()

VILLA                   2468
GROUND_FLOOR            1606
DUPLEX                  1580
MIXED_USE_BUILDING      1240
PENTHOUSE               1170
FLAT_STUDIO              873
EXCEPTIONAL_PROPERTY     566
SERVICE_FLAT             500
MANSION                  479
TOWN_HOUSE               383
COUNTRY_COTTAGE          292
LOFT                     271
BUNGALOW                 212
FARMHOUSE                171
TRIPLEX                   99
KOT                       95
CHALET                    94
MANOR_HOUSE               62
CASTLE                    50
Name: property sub-type, dtype: int64

# Data Cleaning

### Building condition

In [9]:
df['Building condition'].value_counts()
df['Building condition'].unique()

array(['Good', nan, 'As new', 'To renovate', 'To be done up',
       'Just renovated', 'To restore'], dtype=object)

In [10]:
building_condition_map = {'As new': 6, 'Just renovated': 5, 'Good': 4, 'To be done up': 3, 'To renovate':2, 'To restore':1}
df = df.applymap(lambda s: building_condition_map.get(s) if s in building_condition_map else s)

In [11]:
df['Building condition'].isnull().sum()

3828

In [12]:
df['Building condition'] = df['Building condition'].fillna(2)
df['Building condition'].isnull().sum()

0

### Property type & Post code

In [13]:
# Copying the original Property type and Post code column values in new columns
df['Prop_orig'] = df['Property type']
df['Post_code_orig'] = df['Post code']
df = pd.get_dummies(df, columns=['Property type', 'Post code'])

### Kitchen type	


In [14]:
Kit_type_dict = {"USA uninstalled" : 0, 
                 "Not installed" : 0, 
                 "Installed": 1, 
                 "USA installed": 1,
                 "Semi equipped": 1,
                 "USA semi equipped": 1,
                 "Hyper equipped": 2,
                 "USA hyper equipped": 2
                }

df = df.replace(Kit_type_dict)
df["Kitchen type"] = df["Kitchen type"].fillna(0)


df

,Immoweb ID,property sub-type,Price,Building condition,Kitchen type,Bedrooms,Furnished,Terrace surface,Tenement building,Number of frontages,...,Post code_9960,Post code_9968,Post code_9970,Post code_9971,Post code_9981,Post code_9982,Post code_9988,Post code_9990,Post code_9991,Post code_9992
0,9729720,PENTHOUSE,179000.0,4.0,1.0,1.0,Yes,9.0,No,NaN,...,0,0,0,0,0,0,0,0,0,0
1,9729785,PENTHOUSE,255000.0,4.0,1.0,1.0,Yes,46.0,No,2.0,...,0,0,0,0,0,0,0,0,0,0
2,9729784,PENTHOUSE,255000.0,4.0,1.0,1.0,Yes,46.0,No,2.0,...,0,0,0,0,0,0,0,0,0,0
3,9729780,PENTHOUSE,620000.0,4.0,1.0,3.0,Yes,70.0,No,2.0,...,0,0,0,0,0,0,0,0,0,0
4,9727201,PENTHOUSE,379000.0,4.0,2.0,2.0,No,110.0,No,3.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14536,7770552,SERVICE_FLAT,132000.0,6.0,0.0,1.0,NaN,NaN,No,NaN,...,0,0,0,0,0,0,0,0,0,0
14537,7770553,SERVICE_FLAT,177000.0,6.0,0.0,1.0,NaN,NaN,No,NaN,...,0,0,0,0,0,0,0,0,0,0
14538,7770551,SERVICE_FLAT,190000.0,6.0,0.0,2.0,NaN,NaN,No,NaN,...,0,0,0,0,0,0,0,0,0,0
14539,7921693,SERVICE_FLAT,1350000.0,2.0,0.0,30.0,NaN,NaN,No,NaN,...,0,0,0,0,0,0,0,0,0,0


### Furnished


In [15]:
df['Furnished'].unique()

array(['Yes', 'No', nan], dtype=object)

In [16]:
df['Furnished'].isnull().sum()

4080

#### fill No to missing value then map to 1,0

In [17]:
df['Furnished'] = df['Furnished'].fillna("No")
df['Furnished'] = df['Furnished'].apply(lambda v: 0 if v == "No" else 1)

#### double check if value is 1, 0 and no missing value

In [18]:
print(df['Furnished'].unique())
print(df['Furnished'].isna().sum())

[1 0]
0


### Bedrooms


In [19]:
# Fill missing values with 2 bedrooms
df['Bedrooms'] = df['Bedrooms'].fillna(2).astype(int)

df["Bedrooms"].value_counts()

2      4389
3      2929
4      1723
1      1350
5       909
6       429
7       144
8       126
10       61
9        49
12       21
11       20
14       11
13        8
18        7
16        5
17        5
15        5
30        4
25        3
23        3
50        2
60        1
204       1
20        1
22        1
28        1
27        1
35        1
24        1
Name: Bedrooms, dtype: int64

### Terrace surface + Terrace	


#### Combine terrace & terrace surface in yes and no

In [20]:
df['Terrace_Combined'] = np.where(df['Terrace surface'].isnull() & df['Terrace'].isnull(), 'No', 'Yes')

#### Transform Terrace_Combined to 0 & 1

In [21]:
terrace = ['No', 'Yes']

ordenc = OrdinalEncoder(categories=[terrace])

df[["Terrace_Combined"]] = ordenc.fit_transform(df[["Terrace_Combined"]])

df

,Immoweb ID,property sub-type,Price,Building condition,Kitchen type,Bedrooms,Furnished,Terrace surface,Tenement building,Number of frontages,...,Post code_9968,Post code_9970,Post code_9971,Post code_9981,Post code_9982,Post code_9988,Post code_9990,Post code_9991,Post code_9992,Terrace_Combined
0,9729720,PENTHOUSE,179000.0,4.0,1.0,1,1,9.0,No,NaN,...,0,0,0,0,0,0,0,0,0,1.0
1,9729785,PENTHOUSE,255000.0,4.0,1.0,1,1,46.0,No,2.0,...,0,0,0,0,0,0,0,0,0,1.0
2,9729784,PENTHOUSE,255000.0,4.0,1.0,1,1,46.0,No,2.0,...,0,0,0,0,0,0,0,0,0,1.0
3,9729780,PENTHOUSE,620000.0,4.0,1.0,3,1,70.0,No,2.0,...,0,0,0,0,0,0,0,0,0,1.0
4,9727201,PENTHOUSE,379000.0,4.0,2.0,2,0,110.0,No,3.0,...,0,0,0,0,0,0,0,0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14536,7770552,SERVICE_FLAT,132000.0,6.0,0.0,1,0,NaN,No,NaN,...,0,0,0,0,0,0,0,0,0,0.0
14537,7770553,SERVICE_FLAT,177000.0,6.0,0.0,1,0,NaN,No,NaN,...,0,0,0,0,0,0,0,0,0,1.0
14538,7770551,SERVICE_FLAT,190000.0,6.0,0.0,2,0,NaN,No,NaN,...,0,0,0,0,0,0,0,0,0,1.0
14539,7921693,SERVICE_FLAT,1350000.0,2.0,0.0,30,0,NaN,No,NaN,...,0,0,0,0,0,0,0,0,0,0.0


#### Drop Terrace and Terrace Surface columns

In [22]:
df = df.drop(columns =['Terrace', 'Terrace surface'])

df

,Immoweb ID,property sub-type,Price,Building condition,Kitchen type,Bedrooms,Furnished,Tenement building,Number of frontages,Swimming pool,...,Post code_9968,Post code_9970,Post code_9971,Post code_9981,Post code_9982,Post code_9988,Post code_9990,Post code_9991,Post code_9992,Terrace_Combined
0,9729720,PENTHOUSE,179000.0,4.0,1.0,1,1,No,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1.0
1,9729785,PENTHOUSE,255000.0,4.0,1.0,1,1,No,2.0,NaN,...,0,0,0,0,0,0,0,0,0,1.0
2,9729784,PENTHOUSE,255000.0,4.0,1.0,1,1,No,2.0,NaN,...,0,0,0,0,0,0,0,0,0,1.0
3,9729780,PENTHOUSE,620000.0,4.0,1.0,3,1,No,2.0,NaN,...,0,0,0,0,0,0,0,0,0,1.0
4,9727201,PENTHOUSE,379000.0,4.0,2.0,2,0,No,3.0,No,...,0,0,0,0,0,0,0,0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14536,7770552,SERVICE_FLAT,132000.0,6.0,0.0,1,0,No,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0
14537,7770553,SERVICE_FLAT,177000.0,6.0,0.0,1,0,No,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1.0
14538,7770551,SERVICE_FLAT,190000.0,6.0,0.0,2,0,No,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1.0
14539,7921693,SERVICE_FLAT,1350000.0,2.0,0.0,30,0,No,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0


### ~~Tenement building	~~


In [23]:
df = df.drop(columns =['Tenement building'])

df

,Immoweb ID,property sub-type,Price,Building condition,Kitchen type,Bedrooms,Furnished,Number of frontages,Swimming pool,How many fireplaces?,...,Post code_9968,Post code_9970,Post code_9971,Post code_9981,Post code_9982,Post code_9988,Post code_9990,Post code_9991,Post code_9992,Terrace_Combined
0,9729720,PENTHOUSE,179000.0,4.0,1.0,1,1,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1.0
1,9729785,PENTHOUSE,255000.0,4.0,1.0,1,1,2.0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1.0
2,9729784,PENTHOUSE,255000.0,4.0,1.0,1,1,2.0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1.0
3,9729780,PENTHOUSE,620000.0,4.0,1.0,3,1,2.0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1.0
4,9727201,PENTHOUSE,379000.0,4.0,2.0,2,0,3.0,No,NaN,...,0,0,0,0,0,0,0,0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14536,7770552,SERVICE_FLAT,132000.0,6.0,0.0,1,0,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0
14537,7770553,SERVICE_FLAT,177000.0,6.0,0.0,1,0,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1.0
14538,7770551,SERVICE_FLAT,190000.0,6.0,0.0,2,0,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1.0
14539,7921693,SERVICE_FLAT,1350000.0,2.0,0.0,30,0,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0


### Swimming pool	


In [24]:
# Fill missing values with value 0
df['Swimming pool'].fillna(0, inplace = True)
df['Swimming pool'] = df['Swimming pool'].apply(lambda v: 0 if v == "No" else 1)
df['Swimming pool'].isnull().sum()

0

In [25]:
df['Swimming pool'].unique()

array([1, 0])

### Garden + Garden surface	


In [26]:
print(df['Garden'].isna().sum())
print(df['Garden'].unique())

11051
[nan 'Yes']


In [27]:
df.loc[df['Garden surface'].isna()]

,Immoweb ID,property sub-type,Price,Building condition,Kitchen type,Bedrooms,Furnished,Number of frontages,Swimming pool,How many fireplaces?,...,Post code_9968,Post code_9970,Post code_9971,Post code_9981,Post code_9982,Post code_9988,Post code_9990,Post code_9991,Post code_9992,Terrace_Combined
0,9729720,PENTHOUSE,179000.0,4.0,1.0,1,1,NaN,1,NaN,...,0,0,0,0,0,0,0,0,0,1.0
1,9729785,PENTHOUSE,255000.0,4.0,1.0,1,1,2.0,1,NaN,...,0,0,0,0,0,0,0,0,0,1.0
2,9729784,PENTHOUSE,255000.0,4.0,1.0,1,1,2.0,1,NaN,...,0,0,0,0,0,0,0,0,0,1.0
3,9729780,PENTHOUSE,620000.0,4.0,1.0,3,1,2.0,1,NaN,...,0,0,0,0,0,0,0,0,0,1.0
4,9727201,PENTHOUSE,379000.0,4.0,2.0,2,0,3.0,0,NaN,...,0,0,0,0,0,0,0,0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14536,7770552,SERVICE_FLAT,132000.0,6.0,0.0,1,0,NaN,1,NaN,...,0,0,0,0,0,0,0,0,0,0.0
14537,7770553,SERVICE_FLAT,177000.0,6.0,0.0,1,0,NaN,1,NaN,...,0,0,0,0,0,0,0,0,0,1.0
14538,7770551,SERVICE_FLAT,190000.0,6.0,0.0,2,0,NaN,1,NaN,...,0,0,0,0,0,0,0,0,0,1.0
14539,7921693,SERVICE_FLAT,1350000.0,2.0,0.0,30,0,NaN,1,NaN,...,0,0,0,0,0,0,0,0,0,0.0


#### Combine ['Garden'] and ['Garden surface] as ['garden_label] and map to 1,0

In [28]:
def categorise(col):  
    if col['Garden'] == "Yes" or col['Garden surface'] > 0:
        return 1
    return 0


df['garden_label'] = df.apply(lambda col: categorise(col), axis=1)

#### double check if value is 1, 0 and no missing value

In [29]:
df['garden_label'].unique()

array([0, 1])

#### drop column Garden and Garden surface

In [30]:
df = df.drop(columns =['Garden', 'Garden surface'])

df

,Immoweb ID,property sub-type,Price,Building condition,Kitchen type,Bedrooms,Furnished,Number of frontages,Swimming pool,How many fireplaces?,...,Post code_9970,Post code_9971,Post code_9981,Post code_9982,Post code_9988,Post code_9990,Post code_9991,Post code_9992,Terrace_Combined,garden_label
0,9729720,PENTHOUSE,179000.0,4.0,1.0,1,1,NaN,1,NaN,...,0,0,0,0,0,0,0,0,1.0,0
1,9729785,PENTHOUSE,255000.0,4.0,1.0,1,1,2.0,1,NaN,...,0,0,0,0,0,0,0,0,1.0,0
2,9729784,PENTHOUSE,255000.0,4.0,1.0,1,1,2.0,1,NaN,...,0,0,0,0,0,0,0,0,1.0,0
3,9729780,PENTHOUSE,620000.0,4.0,1.0,3,1,2.0,1,NaN,...,0,0,0,0,0,0,0,0,1.0,0
4,9727201,PENTHOUSE,379000.0,4.0,2.0,2,0,3.0,0,NaN,...,0,0,0,0,0,0,0,0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14536,7770552,SERVICE_FLAT,132000.0,6.0,0.0,1,0,NaN,1,NaN,...,0,0,0,0,0,0,0,0,0.0,0
14537,7770553,SERVICE_FLAT,177000.0,6.0,0.0,1,0,NaN,1,NaN,...,0,0,0,0,0,0,0,0,1.0,0
14538,7770551,SERVICE_FLAT,190000.0,6.0,0.0,2,0,NaN,1,NaN,...,0,0,0,0,0,0,0,0,1.0,0
14539,7921693,SERVICE_FLAT,1350000.0,2.0,0.0,30,0,NaN,1,NaN,...,0,0,0,0,0,0,0,0,0.0,0


### ~~Garden orientation~~


#### Drop garden orientation

In [31]:
df = df.drop(columns =['Garden orientation'])

df

,Immoweb ID,property sub-type,Price,Building condition,Kitchen type,Bedrooms,Furnished,Number of frontages,Swimming pool,How many fireplaces?,...,Post code_9970,Post code_9971,Post code_9981,Post code_9982,Post code_9988,Post code_9990,Post code_9991,Post code_9992,Terrace_Combined,garden_label
0,9729720,PENTHOUSE,179000.0,4.0,1.0,1,1,NaN,1,NaN,...,0,0,0,0,0,0,0,0,1.0,0
1,9729785,PENTHOUSE,255000.0,4.0,1.0,1,1,2.0,1,NaN,...,0,0,0,0,0,0,0,0,1.0,0
2,9729784,PENTHOUSE,255000.0,4.0,1.0,1,1,2.0,1,NaN,...,0,0,0,0,0,0,0,0,1.0,0
3,9729780,PENTHOUSE,620000.0,4.0,1.0,3,1,2.0,1,NaN,...,0,0,0,0,0,0,0,0,1.0,0
4,9727201,PENTHOUSE,379000.0,4.0,2.0,2,0,3.0,0,NaN,...,0,0,0,0,0,0,0,0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14536,7770552,SERVICE_FLAT,132000.0,6.0,0.0,1,0,NaN,1,NaN,...,0,0,0,0,0,0,0,0,0.0,0
14537,7770553,SERVICE_FLAT,177000.0,6.0,0.0,1,0,NaN,1,NaN,...,0,0,0,0,0,0,0,0,1.0,0
14538,7770551,SERVICE_FLAT,190000.0,6.0,0.0,2,0,NaN,1,NaN,...,0,0,0,0,0,0,0,0,1.0,0
14539,7921693,SERVICE_FLAT,1350000.0,2.0,0.0,30,0,NaN,1,NaN,...,0,0,0,0,0,0,0,0,0.0,0


### ~~How many fireplaces?	~~


#### Discard fireplaces

In [32]:
df = df.drop(columns =["How many fireplaces?"])

df

,Immoweb ID,property sub-type,Price,Building condition,Kitchen type,Bedrooms,Furnished,Number of frontages,Swimming pool,Surface of the plot,...,Post code_9970,Post code_9971,Post code_9981,Post code_9982,Post code_9988,Post code_9990,Post code_9991,Post code_9992,Terrace_Combined,garden_label
0,9729720,PENTHOUSE,179000.0,4.0,1.0,1,1,NaN,1,NaN,...,0,0,0,0,0,0,0,0,1.0,0
1,9729785,PENTHOUSE,255000.0,4.0,1.0,1,1,2.0,1,NaN,...,0,0,0,0,0,0,0,0,1.0,0
2,9729784,PENTHOUSE,255000.0,4.0,1.0,1,1,2.0,1,NaN,...,0,0,0,0,0,0,0,0,1.0,0
3,9729780,PENTHOUSE,620000.0,4.0,1.0,3,1,2.0,1,NaN,...,0,0,0,0,0,0,0,0,1.0,0
4,9727201,PENTHOUSE,379000.0,4.0,2.0,2,0,3.0,0,NaN,...,0,0,0,0,0,0,0,0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14536,7770552,SERVICE_FLAT,132000.0,6.0,0.0,1,0,NaN,1,NaN,...,0,0,0,0,0,0,0,0,0.0,0
14537,7770553,SERVICE_FLAT,177000.0,6.0,0.0,1,0,NaN,1,NaN,...,0,0,0,0,0,0,0,0,1.0,0
14538,7770551,SERVICE_FLAT,190000.0,6.0,0.0,2,0,NaN,1,NaN,...,0,0,0,0,0,0,0,0,1.0,0
14539,7921693,SERVICE_FLAT,1350000.0,2.0,0.0,30,0,NaN,1,NaN,...,0,0,0,0,0,0,0,0,0.0,0


### Surface of the plot




In [33]:
# Fill empty values with 0
df['Surface of the plot'].fillna(0, inplace = True)

### Living area

#### Fill missing values in Living area row

In [34]:
def fill_living_area(col):  
    if col['Living area'] > 0:
        return col['Living area']
    else: 
        if col['Prop_orig'] == 'APARTMENT':
            return 95.0
        else:
            return 150.0


df['Living area'] = df.apply(lambda col: fill_living_area(col), axis=1)


In [35]:
# Test if there are null values in living area
df['Living area'].isnull().sum()

0

### Price per square metre

In [36]:
df['price/m2'] = (df['Price']/ df['Living area']).round(2)

df

,Immoweb ID,property sub-type,Price,Building condition,Kitchen type,Bedrooms,Furnished,Number of frontages,Swimming pool,Surface of the plot,...,Post code_9971,Post code_9981,Post code_9982,Post code_9988,Post code_9990,Post code_9991,Post code_9992,Terrace_Combined,garden_label,price/m2
0,9729720,PENTHOUSE,179000.0,4.0,1.0,1,1,NaN,1,0.0,...,0,0,0,0,0,0,0,1.0,0,2841.27
1,9729785,PENTHOUSE,255000.0,4.0,1.0,1,1,2.0,1,0.0,...,0,0,0,0,0,0,0,1.0,0,6071.43
2,9729784,PENTHOUSE,255000.0,4.0,1.0,1,1,2.0,1,0.0,...,0,0,0,0,0,0,0,1.0,0,6071.43
3,9729780,PENTHOUSE,620000.0,4.0,1.0,3,1,2.0,1,0.0,...,0,0,0,0,0,0,0,1.0,0,4133.33
4,9727201,PENTHOUSE,379000.0,4.0,2.0,2,0,3.0,0,0.0,...,0,0,0,0,0,0,0,1.0,0,3158.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14536,7770552,SERVICE_FLAT,132000.0,6.0,0.0,1,0,NaN,1,0.0,...,0,0,0,0,0,0,0,0.0,0,1389.47
14537,7770553,SERVICE_FLAT,177000.0,6.0,0.0,1,0,NaN,1,0.0,...,0,0,0,0,0,0,0,1.0,0,1863.16
14538,7770551,SERVICE_FLAT,190000.0,6.0,0.0,2,0,NaN,1,0.0,...,0,0,0,0,0,0,0,1.0,0,2000.00
14539,7921693,SERVICE_FLAT,1350000.0,2.0,0.0,30,0,NaN,1,0.0,...,0,0,0,0,0,0,0,0.0,0,900.00


In [37]:
# Test if there are null values in price/m2 column
df['price/m2'].isnull().sum()

0

### Frontages

In [38]:
df['Number of frontages'].isnull().sum()

3927

In [39]:
#get ['number of frontages'] with values and calc mean
selected_rows = df[~df['Number of frontages'].isnull()]
mean_num_of_frontages = selected_rows['Number of frontages'].mean(axis=0).round(0)
mean_num_of_frontages

3.0

In [40]:
# fill mean value to missing value
df['Number of frontages'] = df['Number of frontages'].fillna(mean_num_of_frontages)
df['Number of frontages'].isnull().sum()

0

In [41]:
df['Number of frontages'] = df['Number of frontages'].astype(int)

In [42]:
# double check
df['Number of frontages'].value_counts()

3     5260
4     3504
2     3346
1       91
6        6
7        2
5        1
16       1
Name: Number of frontages, dtype: int64

### Final check missing value %




In [43]:
percent_missing = df.isnull().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'percent_missing': percent_missing})
missing_value_df

,column_name,percent_missing
Immoweb ID,Immoweb ID,0.0
property sub-type,property sub-type,0.0
Price,Price,0.0
Building condition,Building condition,0.0
Kitchen type,Kitchen type,0.0
...,...,...
Post code_9991,Post code_9991,0.0
Post code_9992,Post code_9992,0.0
Terrace_Combined,Terrace_Combined,0.0
garden_label,garden_label,0.0


# Data Analysis

### Group Post code to region

In [44]:
%pip install pgeocode --quiet

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/opt/homebrew/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [45]:
import pgeocode
nomi = pgeocode.Nominatim("be")
nomi.query_postal_code("1040")['place_name']

'Etterbeek'

In [46]:
nomi.query_postal_code("5030")['state_name']

'Wallonie'

In [47]:
def get_state(col):  
    region = nomi.query_postal_code(col['Post_code_orig'])['state_name']
    if region == "Wallonie":
        region = "Wallonia"
    elif region == "Vlaanderen":
        region = "Flanders"
    else:
        region = "Brussels"
    return region

def get_commune(col):  
    return nomi.query_postal_code(col['Post_code_orig'])['place_name']

df['region'] = df.apply(lambda col: get_state(col), axis=1)
df['commune'] = df.apply(lambda col: get_commune(col), axis=1)


In [48]:
df[['commune', 'Post_code_orig', 'region']]

,commune,Post_code_orig,region
0,Evere,1140,Brussels
1,"Uitkerke, Blankenberge",8370,Flanders
2,"Uitkerke, Blankenberge",8370,Flanders
3,"Uitkerke, Blankenberge",8370,Flanders
4,Laeken,1020,Brussels
...,...,...,...
14536,"Asse, Kobbegem, Bekkerzeel, Mollem",1730,Flanders
14537,"Asse, Kobbegem, Bekkerzeel, Mollem",1730,Flanders
14538,"Asse, Kobbegem, Bekkerzeel, Mollem",1730,Flanders
14539,"Jamiolle, Vodecée, Philippeville, Villers-En-F...",5600,Wallonia


### Most & Least Expensive Municipalities

### Belgium

#### Most expensive municipalities in Belgium based on average price

In [49]:
avg_price_be = df.groupby('Post_code_orig')['Price'].mean()

avg_price_be.sort_values(ascending=False).head(10)

Post_code_orig
5520    5.500000e+06
1404    3.200000e+06
7504    2.350000e+06
8554    2.308333e+06
3080    1.872700e+06
2321    1.659667e+06
1640    1.606700e+06
6662    1.599500e+06
1380    1.589976e+06
9111    1.553333e+06
Name: Price, dtype: float64

#### Most expensive municipalities in Belgium based on median price

In [50]:
median_price_be = df.groupby('Post_code_orig')['Price'].median()

median_price_be.sort_values(ascending=False).head(10)

Post_code_orig
5520    5500000.0
8554    3300000.0
1404    3200000.0
7504    2350000.0
7133    1900000.0
6662    1599500.0
4770    1500000.0
1342    1500000.0
3080    1495000.0
1640    1472500.0
Name: Price, dtype: float64

#### Most expensive municipalities in Belgium based on price/m2

In [51]:
price_m2_be = df.groupby('Post_code_orig')['price/m2'].mean()

price_m2_be.sort_values(ascending=False).head(10)

Post_code_orig
8554    10366.666667
8300     9428.955939
5561     9000.000000
8301     7247.717667
4050     7219.445000
9772     6966.670000
3000     6079.703964
3660     6018.285000
3950     5996.895000
2030     5927.190000
Name: price/m2, dtype: float64

#### Least expensive municipalities in Belgium based on average price

In [52]:
avg_price_be = df.groupby('Post_code_orig')['Price'].mean()

avg_price_be.sort_values(ascending=True).head(10)

Post_code_orig
6592     25000.0
7603     85000.0
5503     90000.0
5570     94000.0
5363    100000.0
8851    110000.0
7784    111287.0
6534    120000.0
6860    122500.0
3631    140000.0
Name: Price, dtype: float64

#### Least expensive municipalities in Belgium based on median price

In [53]:
median_price_be = df.groupby('Post_code_orig')['Price'].median()

median_price_be.sort_values(ascending=True).head(10)

Post_code_orig
6592     25000.0
5540     77000.0
7603     85000.0
5503     90000.0
5570     94000.0
5363    100000.0
8851    110000.0
7784    110787.0
6534    120000.0
6860    122500.0
Name: Price, dtype: float64

#### Least expensive municipalities in Belgium based on price/m2

In [54]:
price_m2_be = df.groupby('Post_code_orig')['price/m2'].mean()

price_m2_be.sort_values(ascending=True).head(10)

Post_code_orig
6592    166.670
6750    306.670
6723    438.600
7504    500.000
7603    500.000
4770    576.920
6673    605.650
5570    626.670
5363    666.670
6860    740.075
Name: price/m2, dtype: float64

### Wallonia

#### Most expensive municipalities in Wallonia based on average price

In [55]:
df_g = df.query('region == "Wallonia"')

df_g = df_g.filter(['Post_code_orig', 'Price'])

avg_price_wal = df_g.groupby('Post_code_orig').mean()

avg_price_wal.sort_values(by=['Price'], ascending=False).head(10)

,Price
Post_code_orig,
5520,5.500000e+06
1404,3.200000e+06
7504,2.350000e+06
6662,1.599500e+06
1380,1.589976e+06
4770,1.500000e+06
7133,1.386667e+06
5561,1.350000e+06
1342,1.237250e+06


#### Most expensive municipalities in Wallonia based on median price

In [56]:
median_price_wal = df_g.groupby('Post_code_orig').median()

median_price_wal.sort_values(by=['Price'], ascending=False).head(10)

,Price
Post_code_orig,
5520,5500000.0
1404,3200000.0
7504,2350000.0
7133,1900000.0
6662,1599500.0
4770,1500000.0
1342,1500000.0
5561,1350000.0
5354,1195000.0


#### Most expensive municipalities in Wallonia based on price/m2

In [57]:
df_f = df.query('region == "Wallonia"')

df_f = df_f.filter(['Post_code_orig', 'price/m2'])

price_m2_wal = df_f.groupby('Post_code_orig').mean()

price_m2_wal.sort_values(by=['price/m2'], ascending=False).head(10)

,price/m2
Post_code_orig,
5561,9000.000000
4050,7219.445000
7130,5266.854286
7943,5000.000000
1332,4893.158667
5651,4516.310000
1341,4339.552500
1404,4266.670000
1348,4110.274706


#### Least expensive municipalities in Wallonia based on average price

In [58]:
df_g = df.query('region == "Wallonia"')

df_g = df_g.filter(['Post_code_orig', 'Price'])

avg_price_wal = df_g.groupby('Post_code_orig').mean()

avg_price_wal.sort_values(by=['Price'], ascending=True).head(10)

,Price
Post_code_orig,
6592,25000.000000
7603,85000.000000
5503,90000.000000
5570,94000.000000
5363,100000.000000
7784,111287.000000
6534,120000.000000
6860,122500.000000
6593,142499.500000


#### Least expensive municipalities in Wallonia based on median price

In [59]:
median_price_wal = df_g.groupby('Post_code_orig').median()

median_price_wal.sort_values(by=['Price'], ascending=True).head(10)

,Price
Post_code_orig,
6592,25000.0
5540,77000.0
7603,85000.0
5503,90000.0
5570,94000.0
5363,100000.0
7784,110787.0
6534,120000.0
6860,122500.0


#### Least expensive municipalities in Wallonia based on price/m2

In [60]:
df_f = df.query('region == "Wallonia"')

df_f = df_f.filter(['Post_code_orig', 'price/m2'])

price_m2_wal = df_f.groupby('Post_code_orig').mean()

price_m2_wal.sort_values(by=['price/m2'], ascending=True).head(10)

,price/m2
Post_code_orig,
6592,166.670
6750,306.670
6723,438.600
7603,500.000
7504,500.000
4770,576.920
6673,605.650
5570,626.670
5363,666.670


### Flanders

#### Most expensive municipalities in Flanders based on average price

In [61]:
df_fl = df.query('region == "Flanders"')

df_fl = df_fl.filter(['Post_code_orig', 'Price'])

avg_price_fl = df_fl.groupby('Post_code_orig').mean()

avg_price_fl.sort_values(by=['Price'], ascending=False).head(10)

,Price
Post_code_orig,
8554,2.308333e+06
3080,1.872700e+06
2321,1.659667e+06
1640,1.606700e+06
9111,1.553333e+06
2360,1.492118e+06
3470,1.381875e+06
8300,1.379083e+06
3212,1.349500e+06


#### Most expensive municipalities in Flanders based on median price

In [62]:
median_price_fl = df_fl.groupby('Post_code_orig').median()

median_price_fl.sort_values(by=['Price'], ascending=False).head(10)

,Price
Post_code_orig,
8554,3300000.0
3080,1495000.0
1640,1472500.0
3212,1349500.0
2812,1200000.0
9830,1200000.0
2330,1174500.0
2950,1150000.0
8904,1099500.0


#### Most expensive municipalities in Flanders based on price/m2

In [63]:
df_fl_pm2 = df.query('region == "Flanders"')

df_fl_pm2 = df_fl_pm2.filter(['Post_code_orig', 'price/m2'])

price_m2_fl = df_fl_pm2.groupby('Post_code_orig').mean()

price_m2_fl.sort_values(by=['price/m2'], ascending=False).head(10)

,price/m2
Post_code_orig,
8554,10366.666667
8300,9428.955939
8301,7247.717667
9772,6966.670000
3000,6079.703964
3660,6018.285000
3950,5996.895000
2030,5927.190000
8904,5780.135000


#### Least expensive municipalities in Flanders based on average price

In [64]:
df_fl_le = df.query('region == "Flanders"')

df_fl_le = df_fl_le.filter(['Post_code_orig', 'Price'])

avg_price_fl = df_fl_le.groupby('Post_code_orig').mean()

avg_price_fl.sort_values(by=['Price'], ascending=True).head(10)

,Price
Post_code_orig,
8851,110000.000000
3631,140000.000000
9970,158000.000000
3051,175000.000000
8430,183973.529412
1670,186500.000000
2450,189000.000000
1654,195000.000000
9950,197500.000000


#### Least expensive municipalities in Flanders based on median price

In [65]:
median_price_fl = df_fl_le.groupby('Post_code_orig').median()

median_price_fl.sort_values(by=['Price'], ascending=True).head(10)

,Price
Post_code_orig,
8851,110000.0
3631,140000.0
8430,140000.0
9970,158000.0
3051,175000.0
8434,179000.0
1670,186500.0
2450,189000.0
8620,189000.0


#### Least expensive municipalities in Flanders based on price/m2

In [66]:
df_fl_le_pm2 = df.query('region == "Flanders"')

df_fl_le_pm2 = df_fl_le_pm2.filter(['Post_code_orig', 'price/m2'])

price_m2_fl = df_fl_le_pm2.groupby('Post_code_orig').mean()

price_m2_fl.sort_values(by=['price/m2'], ascending=True).head(10)

,price/m2
Post_code_orig,
8640,773.446667
8902,775.560000
3321,808.220000
3631,933.330000
8650,1023.970000
3791,1104.060000
9451,1111.110000
8851,1157.890000
8972,1195.000000


### Brussels

#### Most expensive municipalities in Brussels based on average price

In [67]:
df_bru = df.query('region == "Brussels"')

df_bru = df_bru.filter(['Post_code_orig', 'Price'])

avg_price_bru = df_bru.groupby('Post_code_orig').mean()

avg_price_bru.sort_values(by=['Price'], ascending=False).head(10)

,Price
Post_code_orig,
1180,1.152908e+06
1150,1.094220e+06
1050,9.272821e+05
1190,8.069917e+05
1040,7.469559e+05
1060,6.796486e+05
1000,6.626572e+05
1170,6.375000e+05
1160,5.546731e+05


#### Most expensive municipalities in Brussels based on median price

In [68]:
median_price_bru = df_bru.groupby('Post_code_orig').median()

median_price_bru.sort_values(by=['Price'], ascending=False).head(10)

,Price
Post_code_orig,
1180,845000.0
1150,784500.0
1040,547000.0
1050,537500.0
1060,500000.0
1170,482500.0
1160,449500.0
1000,427000.0
1190,415000.0


#### Most expensive municipalities in Brussels based on price/m2

In [69]:
df_bru_pm2 = df.query('region == "Brussels"')

df_bru_pm2 = df_bru_pm2.filter(['Post_code_orig', 'price/m2'])

price_m2_bru = df_bru_pm2.groupby('Post_code_orig').mean()

price_m2_bru.sort_values(by=['price/m2'], ascending=False).head(10)

,price/m2
Post_code_orig,
1180,5472.595397
1150,4997.690909
1040,4953.181324
1050,4894.764722
1160,4487.506154
1000,4303.502056
1060,4215.865000
1170,4110.742500
1200,4055.847250


#### Least expensive municipalities in Brussels based on average price

In [70]:
df_bru_le = df.query('region == "Brussels"')

df_bru_le = df_bru_le.filter(['Post_code_orig', 'Price'])

avg_price_bru = df_bru_le.groupby('Post_code_orig').mean()

avg_price_bru.sort_values(by=['Price'], ascending=True).head(10)

,Price
Post_code_orig,
1081,256309.523810
1140,275113.280702
1080,325897.443182
1130,355600.000000
1030,360543.627451
1090,361589.795918
1083,364416.944444
1120,377296.296296
1020,383661.764706


#### Least expensive municipalities in Brussels based on median price

In [71]:
median_price_bru = df_bru_le.groupby('Post_code_orig').median()

median_price_bru.sort_values(by=['Price'], ascending=True).head(10)

,Price
Post_code_orig,
1081,179000.0
1140,230000.0
1080,242000.0
1090,279500.0
1030,288950.0
1120,295000.0
1083,298777.5
1070,315000.0
1210,329000.0


#### Least expensive municipalities in Brussels based on price/m2

In [72]:
df_bru_le_pm2 = df.query('region == "Brussels"')

df_bru_le_pm2 = df_bru_le_pm2.filter(['Post_code_orig', 'price/m2'])

price_m2_bru = df_bru_le_pm2.groupby('Post_code_orig').mean()

price_m2_bru.sort_values(by=['price/m2'], ascending=True).head(10)

,price/m2
Post_code_orig,
1080,2452.043750
1020,2477.574118
1081,2612.088571
1130,2617.884000
1082,2624.932414
1090,2697.586122
1083,2856.315000
1030,2900.370882
1070,2924.017581


### Graphs

In [73]:
#plt.scatter(x = 'Price', y = 'Bedrooms', data =df)